In [1]:
from scripts.graph_utils import create_heterograph
create_heterograph()

Using backend: pytorch


In [1]:
from scripts.model import EntityClassify
from scripts.dataset import HinNodeClassification

import argparse
import numpy as np
import time
import torch as th
import torch.nn as nn
import torch.nn.functional as F

DATASET_PATH_EN = 'graphs/heterograph_en.bin'
DATASET_PATH_ES = 'graphs/heterograph_es.bin'

Using backend: pytorch


In [2]:
language = input("Please input 'en' for English or 'es' for Spanish:")
if language == 'en':
    DATASET_PATH = DATASET_PATH_EN
else:
    DATASET_PATH = DATASET_PATH_ES

Please input 'en' for English or 'es' for Spanish: es


In [5]:
hg = HinNodeClassification(DATASET_PATH)

In [6]:
graph = hg.g
category = hg.category
num_classes = hg.num_classes
train_mask, val_mask, test_mask = hg.get_idx()
labels = graph.nodes[category].data.pop('labels')
h = graph.ndata['h']

In [7]:
hg.g.nodes[category]

NodeSpace(data={'h': tensor([[ 0.0312, -0.0144,  0.0011,  ...,  0.0223, -0.0072,  0.0065],
        [ 0.0206, -0.0122,  0.0175,  ...,  0.0302, -0.0187,  0.0142],
        [ 0.0209, -0.0159,  0.0052,  ...,  0.0258, -0.0112,  0.0078],
        ...,
        [ 0.0431, -0.0145,  0.0035,  ...,  0.0199,  0.0015,  0.0048],
        [ 0.0187, -0.0015, -0.0085,  ..., -0.0022,  0.0005,  0.0097],
        [ 0.0122, -0.0032, -0.0064,  ...,  0.0086, -0.0056, -0.0052]])})

In [8]:
model = EntityClassify(graph,
                           hg.in_dim,
                           num_classes,
                           num_bases=-1,
                           num_hidden_layers=1,
                           dropout= 0.3,
                           use_self_loop=False)

In [9]:
#  optimizer = th.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2norm)
optimizer = th.optim.Adam(model.parameters())

In [ ]:
# training loop
print("start training...")
dur = []
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    if epoch > 5:
        t0 = time.time()
    logits = model(h=h)[category]
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    loss.backward()
    optimizer.step()
    t1 = time.time()

    if epoch > 5:
        dur.append(t1 - t0)
    train_acc = th.sum(logits[train_mask].argmax(dim=1) == labels[train_mask]).item() / len(train_mask)
    val_loss = F.cross_entropy(logits[val_mask], labels[val_mask])
    val_acc = th.sum(logits[val_mask].argmax(dim=1) == labels[val_mask]).item() / len(val_mask)
    print("Epoch {:05d} | Train Acc: {:.4f} | Train Loss: {:.4f} | Valid Acc: {:.4f} | Valid loss: {:.4f} | Time: {:.4f}".
          format(epoch, train_acc, loss.item(), val_acc, val_loss.item(), np.average(dur)))
print()

start training...


D:\Anaconda\envs\base2\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
D:\Anaconda\envs\base2\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Train Acc: 0.5069 | Train Loss: 0.6932 | Valid Acc: 0.4375 | Valid loss: 0.6933 | Time: nan
Epoch 00001 | Train Acc: 0.5625 | Train Loss: 0.6837 | Valid Acc: 0.5000 | Valid loss: 0.6926 | Time: nan
Epoch 00002 | Train Acc: 0.7083 | Train Loss: 0.6704 | Valid Acc: 0.5000 | Valid loss: 0.6861 | Time: nan
Epoch 00003 | Train Acc: 0.7778 | Train Loss: 0.6563 | Valid Acc: 0.5625 | Valid loss: 0.6791 | Time: nan
Epoch 00004 | Train Acc: 0.7986 | Train Loss: 0.6409 | Valid Acc: 0.6250 | Valid loss: 0.6718 | Time: nan
Epoch 00005 | Train Acc: 0.7986 | Train Loss: 0.6242 | Valid Acc: 0.6250 | Valid loss: 0.6729 | Time: nan


In [ ]:
model.eval()
logits = model.forward()[category]
test_loss = F.cross_entropy(logits[test_mask], labels[test_mask])
test_acc = th.sum(logits[test_mask].argmax(dim=1) == labels[test_mask]).item() / len(test_mask)
print("Test Acc: {:.4f} | Test loss: {:.4f}".format(test_acc, test_loss.item()))
print()